# Technical Assessment Test for IHiS 

sss

## Data Preparation

In [329]:
import pandas as pd
import numpy as np
import re

In [330]:
#Reading data from Excel Sheet name "Demographics"
data = pd.read_excel("01_source_files/Technical Test.xls", sheet_name="Demographics")
data.head()

,NRIC,Gender,Age,Marital Status,New NRIC,Coding - Gender,Age Group,Study Number
0,S1510657E,M,39,Divorced,NaN,NaN,NaN,NaN
1,SN5484655,F,68,Single,NaN,NaN,NaN,NaN
2,S7705341S,M,1,Married,NaN,NaN,NaN,NaN
3,SA9787544,M,35,Married,NaN,NaN,NaN,NaN
4,SA7766801,F,91,Married,NaN,NaN,NaN,NaN


In [331]:
# Quick review of the dataset details, types etc
data.info()

#To understand the size of the dataset
data.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117 entries, 0 to 116
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   NRIC             117 non-null    object 
 1   Gender           117 non-null    object 
 2   Age              117 non-null    int64  
 3   Marital Status   117 non-null    object 
 4   New NRIC         0 non-null      float64
 5   Coding - Gender  0 non-null      float64
 6   Age Group        0 non-null      float64
 7   Study Number     0 non-null      float64
dtypes: float64(4), int64(1), object(3)
memory usage: 7.4+ KB


(117, 8)

In [332]:
data.duplicated().sum()

5

In [333]:
data.isna().sum()

NRIC                 0
Gender               0
Age                  0
Marital Status       0
New NRIC           117
Coding - Gender    117
Age Group          117
Study Number       117
dtype: int64

## New NRIC

In [334]:
#Noticed some empty space in between, removed them by using .replace()
data["NRIC"] = data.NRIC.str.replace(" ","")
data.head()

,NRIC,Gender,Age,Marital Status,New NRIC,Coding - Gender,Age Group,Study Number
0,S1510657E,M,39,Divorced,NaN,NaN,NaN,NaN
1,SN5484655,F,68,Single,NaN,NaN,NaN,NaN
2,S7705341S,M,1,Married,NaN,NaN,NaN,NaN
3,SA9787544,M,35,Married,NaN,NaN,NaN,NaN
4,SA7766801,F,91,Married,NaN,NaN,NaN,NaN


In [335]:
nric_list = pd.DataFrame(data["NRIC"])
print(nric_list.shape)
nric_list.head()

(117, 1)


,NRIC
0,S1510657E
1,SN5484655
2,S7705341S
3,SA9787544
4,SA7766801


In [336]:
new_preprocessed_nric = []

for ic in nric_list["NRIC"]:
    if (ic[0] and ic[-1]).isalpha() == True:
        new_preprocessed_nric.append(ic)
    else:

        start_char = []
        end_char = []
        numbers = []

        if (ic[0]).isdigit() == False:
            start_char.append(ic[0])
        if (ic[1]).isdigit() == False:
            end_char.append(ic[1])
        if (ic[2:]).isdigit() == True:
            numbers.append(ic[2:])

        new_preprocessed_nric.append("".join((start_char+numbers+end_char)))        

In [337]:
new_nric_list = pd.DataFrame(new_preprocessed_nric, columns=["NRIC"])
new_nric_list.head()

,NRIC
0,S1510657E
1,S5484655N
2,S7705341S
3,S9787544A
4,S7766801A


In [338]:
data["New NRIC"] = new_nric_list["NRIC"]
data.head()

,NRIC,Gender,Age,Marital Status,New NRIC,Coding - Gender,Age Group,Study Number
0,S1510657E,M,39,Divorced,S1510657E,NaN,NaN,NaN
1,SN5484655,F,68,Single,S5484655N,NaN,NaN,NaN
2,S7705341S,M,1,Married,S7705341S,NaN,NaN,NaN
3,SA9787544,M,35,Married,S9787544A,NaN,NaN,NaN
4,SA7766801,F,91,Married,S7766801A,NaN,NaN,NaN


## Coding - Gender

In [339]:
#Function to label/code Gender Group

def gender_code(gender_cols):
   if gender_cols == "F" :
      return 1
   else:
      return 2


In [340]:
#Apply the function with the return values to Gender Group Column
data["Coding - Gender"] = data["Gender"].apply (lambda gender_cols: gender_code(gender_cols))
data.head()

,NRIC,Gender,Age,Marital Status,New NRIC,Coding - Gender,Age Group,Study Number
0,S1510657E,M,39,Divorced,S1510657E,2,NaN,NaN
1,SN5484655,F,68,Single,S5484655N,1,NaN,NaN
2,S7705341S,M,1,Married,S7705341S,2,NaN,NaN
3,SA9787544,M,35,Married,S9787544A,2,NaN,NaN
4,SA7766801,F,91,Married,S7766801A,1,NaN,NaN


In [341]:
np.sort(data["Coding - Gender"].unique())

array([1, 2])

## Age Group

In [342]:
#Function to define and categories Age Group

def age_group(age_cols):
   if age_cols <= 9 :
      return "G1"
   if age_cols >= 10 and age_cols <= 19:
      return "G2"
   if age_cols >= 20 and age_cols <= 29:
      return "G3"
   if age_cols >= 30 and age_cols <= 39:
      return "G4"  
   if age_cols >= 40 and age_cols <= 49:
      return "G5"
   if age_cols >= 50 and age_cols <= 59:
      return "G6"
   if age_cols >= 60 and age_cols <= 69:
      return "G7"  
   if age_cols >= 70 and age_cols <= 79:
      return "G8"
   if age_cols >= 80 and age_cols <= 89:
      return "G9"
   if age_cols >= 90:
      return "G0"        


In [343]:
#Apply the function with the return values to Age Group Column
data["Age Group"] = data["Age"].apply (lambda age_cols: age_group(age_cols))
data.head()

,NRIC,Gender,Age,Marital Status,New NRIC,Coding - Gender,Age Group,Study Number
0,S1510657E,M,39,Divorced,S1510657E,2,G4,NaN
1,SN5484655,F,68,Single,S5484655N,1,G7,NaN
2,S7705341S,M,1,Married,S7705341S,2,G1,NaN
3,SA9787544,M,35,Married,S9787544A,2,G4,NaN
4,SA7766801,F,91,Married,S7766801A,1,G0,NaN


In [344]:
np.sort(data["Age Group"].unique())

array(['G0', 'G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9'],
      dtype=object)

## Study Number

In [345]:
data["Study Number"] = data["Age Group"] + " - " + data["Coding - Gender"].astype(str)
data.head(10)

,NRIC,Gender,Age,Marital Status,New NRIC,Coding - Gender,Age Group,Study Number
0,S1510657E,M,39,Divorced,S1510657E,2,G4,G4 - 2
1,SN5484655,F,68,Single,S5484655N,1,G7,G7 - 1
2,S7705341S,M,1,Married,S7705341S,2,G1,G1 - 2
3,SA9787544,M,35,Married,S9787544A,2,G4,G4 - 2
4,SA7766801,F,91,Married,S7766801A,1,G0,G0 - 1
5,SI9731219,F,83,Seperated,S9731219I,1,G9,G9 - 1
6,S5303187B,F,24,Married,S5303187B,1,G3,G3 - 1
7,S5050778Z,F,22,Widowed,S5050778Z,1,G3,G3 - 1
8,S8057221N,M,73,Married,S8057221N,2,G8,G8 - 2
9,S1964251W,M,79,Married,S1964251W,2,G8,G8 - 2


## Worksheet: Study Data

### Study Data Worksheet

In [346]:
#Reading data from Excel Sheet name "Study Data"
study_data =  pd.read_excel("01_source_files/Technical Test.xls", sheet_name="Study Data")
study_data.head()

,Study Number,Old NRIC,New NRIC,Gender,Age,Marital Status,Address 1,Address 2,Contact Number,Ethnic Group
0,NaN,S5050778Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,S7133213U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,SQ3747175,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,SD 1637819,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,S9677422V,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [347]:
study_data.info()
print(study_data.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87 entries, 0 to 86
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Study Number    0 non-null      float64
 1   Old NRIC        87 non-null     object 
 2   New NRIC        0 non-null      float64
 3   Gender          0 non-null      float64
 4   Age             0 non-null      float64
 5   Marital Status  0 non-null      float64
 6   Address 1       0 non-null      float64
 7   Address 2       0 non-null      float64
 8   Contact Number  0 non-null      float64
 9   Ethnic Group    0 non-null      float64
dtypes: float64(9), object(1)
memory usage: 6.9+ KB
(87, 10)


### Extra Information Worksheet

In [348]:
#Reading data from Excel Sheet name "Extra information"
data_extra =  pd.read_excel("01_source_files/Technical Test.xls", sheet_name="Extra information")
data_extra.head()

,NRIC,Address 1,Address 2,Contact Number,Ethnic Group
0,S5050778Z,Tampines,No 944,85694488,Indian
1,S7133213U,Sengkang,Blk 175,98841614,Malay
2,SQ3747175,Yishun,No 362,90466569,Malay
3,SD 1637819,Ang Mo Kio,Blk 471,98780745,Indian
4,S9677422V,Bishan,Blk 272,94541935,Malay


In [349]:
data_extra.info()
print(data_extra.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87 entries, 0 to 86
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   NRIC            87 non-null     object
 1   Address 1       87 non-null     object
 2   Address 2       87 non-null     object
 3   Contact Number  87 non-null     int64 
 4   Ethnic Group    87 non-null     object
dtypes: int64(1), object(4)
memory usage: 3.5+ KB
(87, 5)


In [350]:
data_extra["Contact Number"] = data_extra["Contact Number"].astype(str)

### Combination of Worksheets for Study Data

In [351]:
#Merging/combine the data in Demographics worksheet and Extra Information worksheet
demo_n_extra = data.merge(data_extra, how ="left", on="NRIC")
demo_n_extra.head(10)

,NRIC,Gender,Age,Marital Status,New NRIC,Coding - Gender,Age Group,Study Number,Address 1,Address 2,Contact Number,Ethnic Group
0,S1510657E,M,39,Divorced,S1510657E,2,G4,G4 - 2,Yishun,No 572,95378842,Malay
1,SN5484655,F,68,Single,S5484655N,1,G7,G7 - 1,Bishan,Blk 124,96041021,Malay
2,S7705341S,M,1,Married,S7705341S,2,G1,G1 - 2,Tampines,No 236,92429712,Chinese
3,SA9787544,M,35,Married,S9787544A,2,G4,G4 - 2,NaN,NaN,NaN,NaN
4,SA7766801,F,91,Married,S7766801A,1,G0,G0 - 1,NaN,NaN,NaN,NaN
5,SI9731219,F,83,Seperated,S9731219I,1,G9,G9 - 1,NaN,NaN,NaN,NaN
6,S5303187B,F,24,Married,S5303187B,1,G3,G3 - 1,NaN,NaN,NaN,NaN
7,S5050778Z,F,22,Widowed,S5050778Z,1,G3,G3 - 1,Tampines,No 944,85694488,Indian
8,S8057221N,M,73,Married,S8057221N,2,G8,G8 - 2,Tampines,No 138,87835222,Chinese
9,S1964251W,M,79,Married,S1964251W,2,G8,G8 - 2,Bishan,Blk 852,86784318,Malay


In [352]:
print(demo_n_extra.shape)
demo_n_extra.info()

(117, 12)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 117 entries, 0 to 116
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   NRIC             117 non-null    object
 1   Gender           117 non-null    object
 2   Age              117 non-null    int64 
 3   Marital Status   117 non-null    object
 4   New NRIC         117 non-null    object
 5   Coding - Gender  117 non-null    int64 
 6   Age Group        117 non-null    object
 7   Study Number     117 non-null    object
 8   Address 1        72 non-null     object
 9   Address 2        72 non-null     object
 10  Contact Number   72 non-null     object
 11  Ethnic Group     72 non-null     object
dtypes: int64(2), object(10)
memory usage: 11.9+ KB


In [353]:
data_extra[data_extra["NRIC"] == "SA9787544"]

,NRIC,Address 1,Address 2,Contact Number,Ethnic Group


In [354]:
demo_n_extra["Ethnic Group"].isna().sum()

45

In [355]:
demo_n_extra = demo_n_extra.rename(columns={"NRIC":"Old NRIC"})
demo_n_extra

,Old NRIC,Gender,Age,Marital Status,New NRIC,Coding - Gender,Age Group,Study Number,Address 1,Address 2,Contact Number,Ethnic Group
0,S1510657E,M,39,Divorced,S1510657E,2,G4,G4 - 2,Yishun,No 572,95378842,Malay
1,SN5484655,F,68,Single,S5484655N,1,G7,G7 - 1,Bishan,Blk 124,96041021,Malay
2,S7705341S,M,1,Married,S7705341S,2,G1,G1 - 2,Tampines,No 236,92429712,Chinese
3,SA9787544,M,35,Married,S9787544A,2,G4,G4 - 2,NaN,NaN,NaN,NaN
4,SA7766801,F,91,Married,S7766801A,1,G0,G0 - 1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
112,S4596427P,M,84,Single,S4596427P,2,G9,G9 - 2,Ang Mo Kio,Blk 402,99219932,Malay
113,ST1966155,F,58,Seperated,S1966155T,1,G6,G6 - 1,NaN,NaN,NaN,NaN
114,X0960227H,F,72,Single,X0960227H,1,G8,G8 - 1,Ang Mo Kio,Blk 708,98227144,Indian
115,SA7766801,F,91,Married,S7766801A,1,G0,G0 - 1,NaN,NaN,NaN,NaN


### Final Combine to Study Data (Not Yet Complete!!)

In [356]:
#Merge or Join data based on columns name

In [357]:
new_data = study_data.merge(demo_n_extra, how ="right", on="Old NRIC")
new_data.drop(columns=["Study Number_x","New NRIC_x", "Gender_x", "Age_x", "Marital Status_x", "Address 1_x", "Address 2_x", "Contact Number_x", "Ethnic Group_x"], inplace=True)
new_data.head(10)

,Old NRIC,Gender_y,Age_y,Marital Status_y,New NRIC_y,Coding - Gender,Age Group,Study Number_y,Address 1_y,Address 2_y,Contact Number_y,Ethnic Group_y
0,S1510657E,M,39,Divorced,S1510657E,2,G4,G4 - 2,Yishun,No 572,95378842,Malay
1,SN5484655,F,68,Single,S5484655N,1,G7,G7 - 1,Bishan,Blk 124,96041021,Malay
2,S7705341S,M,1,Married,S7705341S,2,G1,G1 - 2,Tampines,No 236,92429712,Chinese
3,SA9787544,M,35,Married,S9787544A,2,G4,G4 - 2,NaN,NaN,NaN,NaN
4,SA7766801,F,91,Married,S7766801A,1,G0,G0 - 1,NaN,NaN,NaN,NaN
5,SI9731219,F,83,Seperated,S9731219I,1,G9,G9 - 1,NaN,NaN,NaN,NaN
6,S5303187B,F,24,Married,S5303187B,1,G3,G3 - 1,NaN,NaN,NaN,NaN
7,S5050778Z,F,22,Widowed,S5050778Z,1,G3,G3 - 1,Tampines,No 944,85694488,Indian
8,S8057221N,M,73,Married,S8057221N,2,G8,G8 - 2,Tampines,No 138,87835222,Chinese
9,S1964251W,M,79,Married,S1964251W,2,G8,G8 - 2,Bishan,Blk 852,86784318,Malay


In [358]:
new_data.shape

(117, 12)

In [359]:
new_data = new_data.rename(columns={"Gender_y":"Gender",
                                    "Age_y": "Age",
                                    "Marital Status_y": "Marital Status",	
                                    "New NRIC_y": "New NRIC",
                                    "Study Number_y": "Study Number",
                                    "Address 1_y": "Address 1",
                                    "Address 2_y": "Address 2",
                                    "Contact Number_y":	"Contact Number",
                                    "Ethnic Group_y": "Ethnic Group"
                                    })
new_data.head()

,Old NRIC,Gender,Age,Marital Status,New NRIC,Coding - Gender,Age Group,Study Number,Address 1,Address 2,Contact Number,Ethnic Group
0,S1510657E,M,39,Divorced,S1510657E,2,G4,G4 - 2,Yishun,No 572,95378842,Malay
1,SN5484655,F,68,Single,S5484655N,1,G7,G7 - 1,Bishan,Blk 124,96041021,Malay
2,S7705341S,M,1,Married,S7705341S,2,G1,G1 - 2,Tampines,No 236,92429712,Chinese
3,SA9787544,M,35,Married,S9787544A,2,G4,G4 - 2,NaN,NaN,NaN,NaN
4,SA7766801,F,91,Married,S7766801A,1,G0,G0 - 1,NaN,NaN,NaN,NaN


In [360]:
new_data = new_data[["Study Number", "Old NRIC", "New NRIC", "Gender", "Age","Age Group", "Marital Status", "Address 1", "Address 2", "Contact Number", "Ethnic Group"]]
new_data

,Study Number,Old NRIC,New NRIC,Gender,Age,Age Group,Marital Status,Address 1,Address 2,Contact Number,Ethnic Group
0,G4 - 2,S1510657E,S1510657E,M,39,G4,Divorced,Yishun,No 572,95378842,Malay
1,G7 - 1,SN5484655,S5484655N,F,68,G7,Single,Bishan,Blk 124,96041021,Malay
2,G1 - 2,S7705341S,S7705341S,M,1,G1,Married,Tampines,No 236,92429712,Chinese
3,G4 - 2,SA9787544,S9787544A,M,35,G4,Married,NaN,NaN,NaN,NaN
4,G0 - 1,SA7766801,S7766801A,F,91,G0,Married,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
112,G9 - 2,S4596427P,S4596427P,M,84,G9,Single,Ang Mo Kio,Blk 402,99219932,Malay
113,G6 - 1,ST1966155,S1966155T,F,58,G6,Seperated,NaN,NaN,NaN,NaN
114,G8 - 1,X0960227H,X0960227H,F,72,G8,Single,Ang Mo Kio,Blk 708,98227144,Indian
115,G0 - 1,SA7766801,S7766801A,F,91,G0,Married,NaN,NaN,NaN,NaN


## Worksheet: Exception List

### List of unique NRIC

In [363]:
# Number of Unique NRIC (New Formatted)
unique_nric = pd.DataFrame(new_data["New NRIC"].unique(), columns=["NRIC"])
unique_nric.index = unique_nric.index + 1
unique_nric = unique_nric.rename_axis("S/N").reset_index()

unique_nric.head(15)

,S/N,NRIC
0,1,S1510657E
1,2,S5484655N
2,3,S7705341S
3,4,S9787544A
4,5,S7766801A
5,6,S9731219I
6,7,S5303187B
7,8,S5050778Z
8,9,S8057221N
9,10,S1964251W


In [364]:
unique_nric_final = pd.DataFrame(unique_nric[["S/N","NRIC"]]).set_index("S/N")
unique_nric_final.head(15)

,NRIC
S/N,
1,S1510657E
2,S5484655N
3,S7705341S
4,S9787544A
5,S7766801A
6,S9731219I
7,S5303187B
8,S5050778Z
9,S8057221N


In [365]:
unique_nric_final.to_excel("02_exported_worksheet/unique_num_nric.xlsx")

### Number of NRIC not found in Extra Information

In [366]:
#Number of NRIC under Demographics that are found in Extra Information workshee.

nric_notin = pd.DataFrame(data[~data["NRIC"].isin(data_extra["NRIC"])])
print(nric_notin.shape)

nric_notin.reset_index(drop=True, inplace=True)
nric_notin.index = nric_notin.index + 1


nric_notin_final = pd.DataFrame(nric_notin[["NRIC"]].rename_axis("S/N"))
nric_notin_final


(45, 8)


,NRIC
S/N,
1,SA9787544
2,SA7766801
3,SI9731219
4,S5303187B
5,SR7638052
6,SH2467824
7,S4157126T
8,SL5774221
9,SY8473437


In [367]:
nric_notin_final.to_excel("02_exported_worksheet/nric_not_found.xlsx")

## Worksheet: Pivot Table

In [ ]:
# demo_n_extra["Age Group"])["Age Group"].value_counts()

In [ ]:
# pd.pivot_table(data=demo_n_extra,index=["Age Group", "Gender"], values=["Marital Status"],aggfunc=np.sum)



In [ ]:
# p_tabel = pd.pivot_table(data=demo_n_extra, index=["Age Group", "Gender"], values=["Marital Status"], aggfunc=np.sum)
# p_tabel

In [393]:
age_group = pd.DataFrame(new_data["Age Group"].value_counts()).T
age_group

,G7,G9,G3,G2,G5,G8,G4,G1,G0,G6
Age Group,16,15,14,13,13,12,11,11,8,4


In [394]:
gender = pd.DataFrame(new_data["Gender"].value_counts()).T
gender

,F,M
Gender,75,42


In [395]:
marital_status = pd.DataFrame(new_data["Marital Status"].value_counts()).T
marital_status

,Single,Married,Seperated,Divorced,Widowed
Marital Status,36,36,17,14,14


In [396]:
pd.concat((age_group, gender)) #.groupby(['Age Group', 'Gender']).first()

,G7,G9,G3,G2,G5,G8,G4,G1,G0,G6,F,M
Age Group,16.0,15.0,14.0,13.0,13.0,12.0,11.0,11.0,8.0,4.0,NaN,NaN
Gender,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.0,42.0
